In [37]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, SaltRemover
from sklearn.feature_selection import VarianceThreshold

In [38]:
df = pd.read_csv("../data/chemtastes_nonpep.csv")

In [21]:
grouped = df.groupby(["class", "taste"]).size().reset_index(name="count")

In [4]:
bitter = grouped[grouped['taste'] == "Bitterness"]
bitter_top12 = bitter.sort_values(by='count', ascending=False).head(12)
sweet = grouped[grouped['taste'] == "Sweetness"]
sweet_top12 = sweet.sort_values(by='count', ascending=False).head(12)
other = grouped[grouped['taste'] == "Other"]
other_top12 = other.sort_values(by='count', ascending=False).head(12)

In [ ]:
def plot_radar(top12):
    total_count = df.shape[0]
    top12["proportion"] = top12["count"] / total_count

    max_proportion = top12["proportion"].max()

    categories = top12["class"].tolist()
    N = len(categories)
    angles = [n / float(N) * 2 * math.pi for n in range(N)]
    angles += angles[:1]

    ax = plt.subplot(111, polar=True)

    values = top12["proportion"].tolist()
    values += values[:1]

    plt.xticks(angles[:-1], categories)

    y_max = max_proportion * 1.2
    yticks = np.linspace(0, y_max, 4)
    ytick_labels = [f"{int(tick * 100)}%" for tick in yticks]
    plt.yticks(yticks, ytick_labels, color="grey")
    plt.ylim(0, y_max)

    ax.plot(angles, values, linewidth=0.7, color="green")
    ax.grid(alpha=0.5)

    plt.savefig("other.png", transparent=True)
    plt.show()

In [ ]:
plot_radar(other_top12)

In [39]:
df

,smiles,taste,superclass,class,subclass
0,Oc1cc2c(cc1O)C1c3ccc(c(c3OCC1(O)C2)O)O,Sweetness,Organoheterocyclic compounds,Benzopyrans,1-benzopyrans
1,CC(C)=CCCC(C)(O)C1CC(O)C(=CC1=O)C,Sweetness,Lipids and lipid-like molecules,Prenol lipids,Sesquiterpenoids
2,CC(=O)OC1C(Oc2cc(cc(c2C1=O)O)O)c1ccc(c(c1)O)O,Sweetness,Phenylpropanoids and polyketides,Flavonoids,Flavans
3,COc1c(cc2c(c1O)C(=O)C(OC(C)=O)C(O2)c1ccc(c(c1)...,Sweetness,Phenylpropanoids and polyketides,Flavonoids,O-methylated flavonoids
4,COc1ccc(cc1O)C1Cc2cccc(c2C(=O)O1)OC1OC(CO)C(O)...,Sweetness,Organic oxygen compounds,Organooxygen compounds,Carbohydrates and carbohydrate conjugates
...,...,...,...,...,...
2979,CC(CCC(O)C(C)(C)O)C1CCC2(C)C3CC=C4C(CCC(O)C4(C...,Non-bitterness,Lipids and lipid-like molecules,Steroids and steroid derivatives,Cucurbitacins
2980,OCC(O)C(O)C(=O)CO,Non-bitterness,Organic oxygen compounds,Organooxygen compounds,Carbohydrates and carbohydrate conjugates
2981,CC1OC(C)C(OS(O)(=O)=O)C(O)C1O,Non-bitterness,Organoheterocyclic compounds,Oxanes,Unknown
2982,CC(=O)OC(C)(C)C1Cc2cc3c(cc2O1)OC(=O)C(=C3)C(C)...,Non-bitterness,Phenylpropanoids and polyketides,Coumarins and derivatives,Furanocoumarins


In [55]:
mols = df["smiles"].apply(Chem.MolFromSmiles)
descs_df = pd.DataFrame([Descriptors.CalcMolDescriptors(mol) for mol in mols])

In [59]:
# Ipc is a 3D descriptor that would cause issues for some large molecules
descs_df.drop(columns="Ipc", inplace=True)

In [60]:
descs_df.shape

(2984, 216)

In [25]:
def dump(content):
    with open("../missing.txt", "w") as f:
        f.write(content)

# Imputation of BCUT2D descriptors

In [61]:
impu_index = descs_df[descs_df["BCUT2D_MWHI"].isna()].index
df.iloc[impu_index]["smiles"]

7       [Na+].NC(CC(O)(CC1=CNc2ccccc12)C(O)=O)C([O-])=O
213                      [Na+].CC(C)(C)CCNS([O-])(=O)=O
440                  [Na+].Clc1cccc2c1C(=O)[N-]S2(=O)=O
468                   [Na+].CCC1=C(C)OS(=O)(=O)[N-]C1=O
472                  [Na+].CC1=C(C)C(=O)[N-]S(=O)(=O)O1
                             ...                       
2955                        [Na+].[O-]S(=O)(=O)N1CCCCC1
2956                    [Na+].Cc1ccc(cc1)NS([O-])(=O)=O
2957                         [Na+].[O-]S(=O)(=O)N1CCCC1
2958          [Na+].CC(=O)Nc1ccc2c(c1)S(=O)(=O)[N-]C2=O
2959                        [Na+].[O-]S(=O)(=O)N1CCSCC1
Name: smiles, Length: 427, dtype: object

In [62]:
def remove_salts(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    remover = SaltRemover.SaltRemover()
    return Chem.MolToSmiles(remover.StripMol(mol))

def calc_bcut2d(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    return rdMolDescriptors.BCUT2D(mol)

In [63]:
bcut2d_descs = df.iloc[impu_index]["smiles"].apply(remove_salts).apply(calc_bcut2d)
bcut2d_cols = ["BCUT2D_MWHI", "BCUT2D_MWLOW", "BCUT2D_CHGHI", "BCUT2D_CHGLO",
               "BCUT2D_LOGPHI", "BCUT2D_LOGPLOW", "BCUT2D_MRHI", "BCUT2D_MRLOW"]
bcut2d_df = pd.DataFrame(bcut2d_descs.tolist(), index=impu_index, columns=bcut2d_cols)

In [64]:
descs_df.loc[bcut2d_df.index, bcut2d_df.columns] = bcut2d_df

In [65]:
descs_df.isna().sum().sum()

np.int64(0)

# Remove constant or near-constant descriptors

In [66]:
selector = VarianceThreshold(threshold=0.001)
filtered = selector.fit_transform(descs_df)
retained_descs = descs_df.columns[selector.get_support()]
vt_df = pd.DataFrame(filtered, columns=retained_descs)

In [67]:
vt_df.shape

(2984, 198)

# Remove highly correlated descriptors

In [68]:
corr_matrix = vt_df.corr().abs()

In [69]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
df_final = vt_df.drop(to_drop, axis=1)

In [70]:
df_final.shape

(2984, 127)

In [71]:
pd.concat([df, df_final], axis=1).to_csv("../data/chemtastes_features_nonpep.csv", index=False)